In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator, img_to_array
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16

In [3]:
TRAIN_DIR = '/content/drive/MyDrive/manufacturing_bigdata/03.deep_learning/workspace/dog_breed/train/'
TEST_DIR = '/content/drive/MyDrive/manufacturing_bigdata/03.deep_learning/workspace/dog_breed/test/'

In [4]:
IM_WIDTH, IM_HEIGHT = 224, 224

In [5]:
train_datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1/255.0
)

In [9]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=100,
    class_mode='categorical',
    shuffle=True
)

Found 8178 images belonging to 120 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1/255.0)

In [10]:
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IM_WIDTH, IM_WIDTH),
    batch_size=100,
    class_mode='categorical',
    shuffle=True
)

Found 2044 images belonging to 120 classes.


In [11]:
conv_layers = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(IM_HEIGHT, IM_WIDTH, 3)
)

58900480/58889256 [==============================] - 0s 0us/step


In [12]:
for layer in conv_layers.layers[:15]:
    layer.trainable = False

In [14]:
model = Sequential()

model.add(conv_layers)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(120, activation='softmax'))

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_3 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 120)              

In [16]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-5), metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [17]:
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/content/drive/MyDrive/Colab Notebooks/DSBA/dental_image/dental_vgg16.h5',
    monitor = 'val_acc',
    vervose = 1,
    save_best_only=True
)

In [ ]:
with tf.device('/device:GPU:0'):
    history = model.fit(
                train_generator,
                epochs=300,
                validation_data=test_generator,
                callbacks=[cb_checkpoint]
                )

Epoch 1/300
80/82 [============================>.] - ETA: 33s - loss: 4.9661 - acc: 0.0089

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['acc'], label='train acc')
plt.plot(history.history['val_loss'], label='val loss')
plt.plot(history.history['val_acc'], label='val acc')
plt.legend(prop={'size': 20}); plt.grid()
plt.ylim(0, 1.0)

In [ ]:
model.evaluate(test_generator)

2/2 [==============================] - 1s 62ms/step - loss: 0.5415 - acc: 0.8167


[0.5415136218070984, 0.8166666626930237]

In [ ]:
import keras

best_model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Git/commit_folder/DSBA/dog_cls_vgg16.h5')

In [ ]:
best_model.evaluate(test_generator)

2/2 [==============================] - 1s 62ms/step - loss: 0.5292 - acc: 0.8500


[0.5291526317596436, 0.8500000238418579]